In [1]:
# Import dependencies
import pandas as pd
pd.set_option('display.max_rows', 10)

In [2]:
# Load raw data files
Latest_Vax_df = pd.read_csv('Covid9_7_2022.csv')
Latest_Deaths_df = pd.read_csv('us-counties-recent.csv')
Lat_Long_df = pd.read_csv('uscounties.csv')
Econ_df = pd.read_csv('unemplymentmedianincome.csv')

In [3]:
### WORKING ON DEATHS DF
# Get only the latests week's data for the deaths dataframe
Latest_deaths_finaldate_filter = Latest_Deaths_df["date"] == Latest_Deaths_df["date"].max()
Latest_Deaths_df = Latest_Deaths_df[Latest_deaths_finaldate_filter]
Latest_Deaths_df

,date,county,state,fips,cases,deaths
94427,2022-09-14,Autauga,Alabama,1001.0,18233,226.0
94428,2022-09-14,Baldwin,Alabama,1003.0,65088,702.0
94429,2022-09-14,Barbour,Alabama,1005.0,6826,101.0
94430,2022-09-14,Bibb,Alabama,1007.0,7450,107.0
94431,2022-09-14,Blount,Alabama,1009.0,16673,254.0
...,...,...,...,...,...,...
97681,2022-09-14,Sweetwater,Wyoming,56037.0,12117,130.0
97682,2022-09-14,Teton,Wyoming,56039.0,11583,16.0
97683,2022-09-14,Uinta,Wyoming,56041.0,6102,41.0
97684,2022-09-14,Washakie,Wyoming,56043.0,2642,47.0


In [4]:
## Replacing the "state" column with a "ST" column containing the state abbreviation
# 1 extract the old column
Latest_Deaths_df["state"]

94427    Alabama
94428    Alabama
94429    Alabama
94430    Alabama
94431    Alabama
          ...   
97681    Wyoming
97682    Wyoming
97683    Wyoming
97684    Wyoming
97685    Wyoming
Name: state, Length: 3259, dtype: object

In [5]:
# 2 create the mapping series
# 3 Use series constructor
States_s = pd.Series(
    Lat_Long_df["state_id"].values, index=Lat_Long_df["state_name"]).drop_duplicates()
States_s

state_name
California       CA
Illinois         IL
Texas            TX
Arizona          AZ
Florida          FL
                 ..
West Virginia    WV
North Dakota     ND
Vermont          VT
Montana          MT
Wyoming          WY
Length: 51, dtype: object

In [6]:
# 4 adjust the code to add the new column to the DataFrame
# 5 Delete the old column from the dataframe
Latest_Deaths_df["ST"] = Latest_Deaths_df["state"].map(States_s)
Latest_Deaths_df.drop(columns="state", inplace=True)
Latest_Deaths_df

,date,county,fips,cases,deaths,ST
94427,2022-09-14,Autauga,1001.0,18233,226.0,AL
94428,2022-09-14,Baldwin,1003.0,65088,702.0,AL
94429,2022-09-14,Barbour,1005.0,6826,101.0,AL
94430,2022-09-14,Bibb,1007.0,7450,107.0,AL
94431,2022-09-14,Blount,1009.0,16673,254.0,AL
...,...,...,...,...,...,...
97681,2022-09-14,Sweetwater,56037.0,12117,130.0,WY
97682,2022-09-14,Teton,56039.0,11583,16.0,WY
97683,2022-09-14,Uinta,56041.0,6102,41.0,WY
97684,2022-09-14,Washakie,56043.0,2642,47.0,WY


In [7]:
## Finding rows with no FIPS value
death_nan_filter = Latest_Deaths_df["fips"].isna()
Latest_Deaths_df[death_nan_filter]

,date,county,fips,cases,deaths,ST
94490,2022-09-14,Unknown,NaN,6912,82.0,AL
94518,2022-09-14,Unknown,NaN,9111,1.0,AK
94523,2022-09-14,Unknown,NaN,8217,34.0,NaN
94609,2022-09-14,Unknown,NaN,31025,0.0,AR
94744,2022-09-14,Unknown,NaN,2849,2.0,CT
...,...,...,...,...,...,...
96839,2022-09-14,Unknown,NaN,32840,5053.0,NaN
96851,2022-09-14,Unknown,NaN,28772,3.0,RI
97052,2022-09-14,Unknown,NaN,22654,270.0,TN
97340,2022-09-14,Unknown,NaN,4972,103.0,UT


In [8]:
# 39 rows isn't too much data; dropping these rows
Latest_Deaths_df = Latest_Deaths_df.dropna(subset=['fips'])
Latest_Deaths_df

,date,county,fips,cases,deaths,ST
94427,2022-09-14,Autauga,1001.0,18233,226.0,AL
94428,2022-09-14,Baldwin,1003.0,65088,702.0,AL
94429,2022-09-14,Barbour,1005.0,6826,101.0,AL
94430,2022-09-14,Bibb,1007.0,7450,107.0,AL
94431,2022-09-14,Blount,1009.0,16673,254.0,AL
...,...,...,...,...,...,...
97681,2022-09-14,Sweetwater,56037.0,12117,130.0,WY
97682,2022-09-14,Teton,56039.0,11583,16.0,WY
97683,2022-09-14,Uinta,56041.0,6102,41.0,WY
97684,2022-09-14,Washakie,56043.0,2642,47.0,WY


In [9]:
## Changing datatype of FIPS column to int so that it can be merged with other datasets
Latest_Deaths_df['fips'] = Latest_Deaths_df['fips'].astype(int)
Latest_Deaths_df

C:\Users\josep\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,date,county,fips,cases,deaths,ST
94427,2022-09-14,Autauga,1001,18233,226.0,AL
94428,2022-09-14,Baldwin,1003,65088,702.0,AL
94429,2022-09-14,Barbour,1005,6826,101.0,AL
94430,2022-09-14,Bibb,1007,7450,107.0,AL
94431,2022-09-14,Blount,1009,16673,254.0,AL
...,...,...,...,...,...,...
97681,2022-09-14,Sweetwater,56037,12117,130.0,WY
97682,2022-09-14,Teton,56039,11583,16.0,WY
97683,2022-09-14,Uinta,56041,6102,41.0,WY
97684,2022-09-14,Washakie,56043,2642,47.0,WY


In [10]:
# Checking for null values
Latest_Deaths_df.isna().sum()

date       0
county     0
fips       0
cases      0
deaths    78
ST        84
dtype: int64

In [11]:
# Dropping the rows without state information
Latest_Deaths_df = Latest_Deaths_df.dropna(subset=['ST'])
Latest_Deaths_df.isna().sum()

date      0
county    0
fips      0
cases     0
deaths    0
ST        0
dtype: int64

In [12]:
# Dropping four counties that are not in Lat Long
Latest_Deaths_df.drop([94499, 94519, 94521, 97247], inplace = True)
Latest_Deaths_df

C:\Users\josep\anaconda3\envs\mlenv\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,date,county,fips,cases,deaths,ST
94427,2022-09-14,Autauga,1001,18233,226.0,AL
94428,2022-09-14,Baldwin,1003,65088,702.0,AL
94429,2022-09-14,Barbour,1005,6826,101.0,AL
94430,2022-09-14,Bibb,1007,7450,107.0,AL
94431,2022-09-14,Blount,1009,16673,254.0,AL
...,...,...,...,...,...,...
97681,2022-09-14,Sweetwater,56037,12117,130.0,WY
97682,2022-09-14,Teton,56039,11583,16.0,WY
97683,2022-09-14,Uinta,56041,6102,41.0,WY
97684,2022-09-14,Washakie,56043,2642,47.0,WY


In [13]:
### Exporting deaths df as it is now finalized
Latest_Deaths_df.to_csv("Latest_Deaths_cleaned.csv", index=False)

In [14]:
### WORKING ON VAX DF
# Renaming "Recip_State" column "ST" to match other dataframes
Latest_Vax_df = Latest_Vax_df.rename(columns={"Recip_State":"ST"})
Latest_Vax_df

,Date,FIPS,MMWR_week,Recip_County,ST,Completeness_pct,Administered_Dose1_Recip,Administered_Dose1_Pop_Pct,Administered_Dose1_Recip_5Plus,Administered_Dose1_Recip_5PlusPop_Pct,...,Booster_Doses_Vax_Pct_UR_Equity,Booster_Doses_12PlusVax_Pct_UR_Equity,Booster_Doses_18PlusVax_Pct_UR_Equity,Booster_Doses_65PlusVax_Pct_UR_Equity,Census2019,Census2019_5PlusPop,Census2019_5to17Pop,Census2019_12PlusPop,Census2019_18PlusPop,Census2019_65PlusPop
0,9/7/2022,1001,36,Autauga County,AL,92.5,31825.0,57.0,31798.0,60.5,...,2.0,2.0,2.0,2.0,55869.0,52592.0,9688.0,47574.0,42904.0,8924.0
1,9/7/2022,1003,36,Baldwin County,AL,92.5,146676.0,65.7,146508.0,69.4,...,2.0,2.0,2.0,2.0,223234.0,211195.0,35515.0,192649.0,175680.0,46830.0
2,9/7/2022,1005,36,Barbour County,AL,92.5,14042.0,56.9,14030.0,60.0,...,6.0,6.0,6.0,6.0,24686.0,23377.0,3773.0,21404.0,19604.0,4861.0
3,9/7/2022,1007,36,Bibb County,AL,92.5,9672.0,43.2,9658.0,45.7,...,2.0,2.0,3.0,2.0,22394.0,21148.0,3311.0,19480.0,17837.0,3733.0
4,9/7/2022,1009,36,Blount County,AL,92.5,22645.0,39.2,22636.0,41.6,...,2.0,2.0,2.0,2.0,57826.0,54388.0,9817.0,49234.0,44571.0,10814.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3219,9/7/2022,72151,36,Yabucoa Municipio,PR,97.3,28112.0,87.1,28068.0,89.9,...,4.0,4.0,4.0,3.0,32282.0,31208.0,4554.0,28791.0,26654.0,7163.0
3220,9/7/2022,72153,36,Yauco Municipio,PR,97.3,33522.0,95.0,33480.0,95.0,...,4.0,4.0,4.0,3.0,33575.0,32383.0,4658.0,29944.0,27725.0,7802.0
3221,9/7/2022,78010,36,St. Croix Island,VI,86.9,27943.0,NaN,27899.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3222,9/7/2022,78020,36,St. John Island,VI,86.9,3935.0,NaN,3934.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# There are 19 counties without a percent vaccinated figure
Latest_Vax_df[Latest_Vax_df['Series_Complete_Pop_Pct'].isna()]

,Date,FIPS,MMWR_week,Recip_County,ST,Completeness_pct,Administered_Dose1_Recip,Administered_Dose1_Pop_Pct,Administered_Dose1_Recip_5Plus,Administered_Dose1_Recip_5PlusPop_Pct,...,Booster_Doses_Vax_Pct_UR_Equity,Booster_Doses_12PlusVax_Pct_UR_Equity,Booster_Doses_18PlusVax_Pct_UR_Equity,Booster_Doses_65PlusVax_Pct_UR_Equity,Census2019,Census2019_5PlusPop,Census2019_5to17Pop,Census2019_12PlusPop,Census2019_18PlusPop,Census2019_65PlusPop
187,9/7/2022,6003,36,Alpine County,CA,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1129.0,1080.0,151.0,1013.0,929.0,298.0
199,9/7/2022,6027,36,Inyo County,CA,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,18039.0,17041.0,2729.0,15521.0,14312.0,4228.0
207,9/7/2022,6043,36,Mariposa County,CA,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,17203.0,16474.0,2049.0,15399.0,14425.0,4974.0
210,9/7/2022,6049,36,Modoc County,CA,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,8841.0,8416.0,1294.0,7730.0,7122.0,2431.0
211,9/7/2022,6051,36,Mono County,CA,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,14444.0,13757.0,1882.0,12770.0,11875.0,2330.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1220,9/7/2022,25007,36,Dukes County,MA,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,17332.0,16489.0,2242.0,15293.0,14247.0,4346.0
1226,9/7/2022,25019,36,Nantucket County,MA,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,11399.0,10647.0,1618.0,9755.0,9029.0,1692.0
3221,9/7/2022,78010,36,St. Croix Island,VI,86.9,27943.0,NaN,27899.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3222,9/7/2022,78020,36,St. John Island,VI,86.9,3935.0,NaN,3934.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# Dropped those 19 rows
Latest_Vax_df = Latest_Vax_df.dropna(subset=['Series_Complete_Pop_Pct'])
Latest_Vax_df

,Date,FIPS,MMWR_week,Recip_County,ST,Completeness_pct,Administered_Dose1_Recip,Administered_Dose1_Pop_Pct,Administered_Dose1_Recip_5Plus,Administered_Dose1_Recip_5PlusPop_Pct,...,Booster_Doses_Vax_Pct_UR_Equity,Booster_Doses_12PlusVax_Pct_UR_Equity,Booster_Doses_18PlusVax_Pct_UR_Equity,Booster_Doses_65PlusVax_Pct_UR_Equity,Census2019,Census2019_5PlusPop,Census2019_5to17Pop,Census2019_12PlusPop,Census2019_18PlusPop,Census2019_65PlusPop
0,9/7/2022,1001,36,Autauga County,AL,92.5,31825.0,57.0,31798.0,60.5,...,2.0,2.0,2.0,2.0,55869.0,52592.0,9688.0,47574.0,42904.0,8924.0
1,9/7/2022,1003,36,Baldwin County,AL,92.5,146676.0,65.7,146508.0,69.4,...,2.0,2.0,2.0,2.0,223234.0,211195.0,35515.0,192649.0,175680.0,46830.0
2,9/7/2022,1005,36,Barbour County,AL,92.5,14042.0,56.9,14030.0,60.0,...,6.0,6.0,6.0,6.0,24686.0,23377.0,3773.0,21404.0,19604.0,4861.0
3,9/7/2022,1007,36,Bibb County,AL,92.5,9672.0,43.2,9658.0,45.7,...,2.0,2.0,3.0,2.0,22394.0,21148.0,3311.0,19480.0,17837.0,3733.0
4,9/7/2022,1009,36,Blount County,AL,92.5,22645.0,39.2,22636.0,41.6,...,2.0,2.0,2.0,2.0,57826.0,54388.0,9817.0,49234.0,44571.0,10814.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3216,9/7/2022,72145,36,Vega Baja Municipio,PR,97.3,50500.0,95.0,50430.0,95.0,...,4.0,4.0,4.0,3.0,50023.0,48014.0,7027.0,44613.0,40987.0,10563.0
3217,9/7/2022,72147,36,Vieques Municipio,PR,97.3,8130.0,95.0,8099.0,95.0,...,8.0,8.0,8.0,7.0,8386.0,8055.0,1211.0,7479.0,6844.0,1895.0
3218,9/7/2022,72149,36,Villalba Municipio,PR,97.3,20978.0,95.0,20910.0,95.0,...,4.0,4.0,4.0,4.0,21372.0,20419.0,3299.0,19061.0,17120.0,3980.0
3219,9/7/2022,72151,36,Yabucoa Municipio,PR,97.3,28112.0,87.1,28068.0,89.9,...,4.0,4.0,4.0,3.0,32282.0,31208.0,4554.0,28791.0,26654.0,7163.0


In [17]:
# Renaming "FIPS" to "fips" to be consistent with other datasets
Latest_Vax_df = Latest_Vax_df.rename(columns={"FIPS":"fips", "Series_Complete_Pop_Pct":"Vax_Pct"})
Latest_Vax_df

,Date,fips,MMWR_week,Recip_County,ST,Completeness_pct,Administered_Dose1_Recip,Administered_Dose1_Pop_Pct,Administered_Dose1_Recip_5Plus,Administered_Dose1_Recip_5PlusPop_Pct,...,Booster_Doses_Vax_Pct_UR_Equity,Booster_Doses_12PlusVax_Pct_UR_Equity,Booster_Doses_18PlusVax_Pct_UR_Equity,Booster_Doses_65PlusVax_Pct_UR_Equity,Census2019,Census2019_5PlusPop,Census2019_5to17Pop,Census2019_12PlusPop,Census2019_18PlusPop,Census2019_65PlusPop
0,9/7/2022,1001,36,Autauga County,AL,92.5,31825.0,57.0,31798.0,60.5,...,2.0,2.0,2.0,2.0,55869.0,52592.0,9688.0,47574.0,42904.0,8924.0
1,9/7/2022,1003,36,Baldwin County,AL,92.5,146676.0,65.7,146508.0,69.4,...,2.0,2.0,2.0,2.0,223234.0,211195.0,35515.0,192649.0,175680.0,46830.0
2,9/7/2022,1005,36,Barbour County,AL,92.5,14042.0,56.9,14030.0,60.0,...,6.0,6.0,6.0,6.0,24686.0,23377.0,3773.0,21404.0,19604.0,4861.0
3,9/7/2022,1007,36,Bibb County,AL,92.5,9672.0,43.2,9658.0,45.7,...,2.0,2.0,3.0,2.0,22394.0,21148.0,3311.0,19480.0,17837.0,3733.0
4,9/7/2022,1009,36,Blount County,AL,92.5,22645.0,39.2,22636.0,41.6,...,2.0,2.0,2.0,2.0,57826.0,54388.0,9817.0,49234.0,44571.0,10814.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3216,9/7/2022,72145,36,Vega Baja Municipio,PR,97.3,50500.0,95.0,50430.0,95.0,...,4.0,4.0,4.0,3.0,50023.0,48014.0,7027.0,44613.0,40987.0,10563.0
3217,9/7/2022,72147,36,Vieques Municipio,PR,97.3,8130.0,95.0,8099.0,95.0,...,8.0,8.0,8.0,7.0,8386.0,8055.0,1211.0,7479.0,6844.0,1895.0
3218,9/7/2022,72149,36,Villalba Municipio,PR,97.3,20978.0,95.0,20910.0,95.0,...,4.0,4.0,4.0,4.0,21372.0,20419.0,3299.0,19061.0,17120.0,3980.0
3219,9/7/2022,72151,36,Yabucoa Municipio,PR,97.3,28112.0,87.1,28068.0,89.9,...,4.0,4.0,4.0,3.0,32282.0,31208.0,4554.0,28791.0,26654.0,7163.0


In [18]:
# Selecting the columns I want
Latest_Vax_df = Latest_Vax_df[["fips","Recip_County","ST","Vax_Pct"]]
Latest_Vax_df

,fips,Recip_County,ST,Vax_Pct
0,1001,Autauga County,AL,45.1
1,1003,Baldwin County,AL,52.2
2,1005,Barbour County,AL,46.7
3,1007,Bibb County,AL,35.8
4,1009,Blount County,AL,32.4
...,...,...,...,...
3216,72145,Vega Baja Municipio,PR,86.4
3217,72147,Vieques Municipio,PR,87.5
3218,72149,Villalba Municipio,PR,91.3
3219,72151,Yabucoa Municipio,PR,77.4


In [19]:
# Dropping counties that are not in Lat Long
Latest_Vax_df = Latest_Vax_df[Latest_Vax_df.ST != "PR"]
Latest_Vax_df = Latest_Vax_df[Latest_Vax_df.ST != "GU"]
Latest_Vax_df.drop([92], inplace = True)
Latest_Vax_df

,fips,Recip_County,ST,Vax_Pct
0,1001,Autauga County,AL,45.1
1,1003,Baldwin County,AL,52.2
2,1005,Barbour County,AL,46.7
3,1007,Bibb County,AL,35.8
4,1009,Blount County,AL,32.4
...,...,...,...,...
3137,56037,Sweetwater County,WY,51.4
3138,56039,Teton County,WY,94.2
3139,56041,Uinta County,WY,52.7
3140,56043,Washakie County,WY,44.8


In [20]:
### Exporting deaths df as it is now finalized
Latest_Vax_df.to_csv("Latest_Vax_cleaned.csv", index=False)

In [21]:
### WORKING ON LAT/LONG dataframe
# Renaming "state_id" to "ST" and "county_fips" to "fips" to be consistent with other datasets
Lat_Long_df = Lat_Long_df.rename(columns={"state_id":"ST", "county_fips":"fips"})
Lat_Long_df

,county,county_ascii,county_full,fips,ST,state_name,lat,lng,population
0,Los Angeles,Los Angeles,Los Angeles County,6037,CA,California,34.3209,-118.2247,10040682
1,Cook,Cook,Cook County,17031,IL,Illinois,41.8401,-87.8168,5169517
2,Harris,Harris,Harris County,48201,TX,Texas,29.8578,-95.3936,4680609
3,Maricopa,Maricopa,Maricopa County,4013,AZ,Arizona,33.3490,-112.4915,4412779
4,San Diego,San Diego,San Diego County,6073,CA,California,33.0343,-116.7350,3323970
...,...,...,...,...,...,...,...,...,...
3138,Harding,Harding,Harding County,35021,NM,New Mexico,35.8579,-103.8202,432
3139,McPherson,McPherson,McPherson County,31117,NE,Nebraska,41.5682,-101.0604,420
3140,Kenedy,Kenedy,Kenedy County,48261,TX,Texas,26.9285,-97.7017,391
3141,King,King,King County,48269,TX,Texas,33.6165,-100.2558,279


In [22]:
## Choosing the columns I want from lat long
Lat_Long_df = Lat_Long_df[["fips","ST","lat","lng","population"]]
Lat_Long_df

,fips,ST,lat,lng,population
0,6037,CA,34.3209,-118.2247,10040682
1,17031,IL,41.8401,-87.8168,5169517
2,48201,TX,29.8578,-95.3936,4680609
3,4013,AZ,33.3490,-112.4915,4412779
4,6073,CA,33.0343,-116.7350,3323970
...,...,...,...,...,...
3138,35021,NM,35.8579,-103.8202,432
3139,31117,NE,41.5682,-101.0604,420
3140,48261,TX,26.9285,-97.7017,391
3141,48269,TX,33.6165,-100.2558,279


In [23]:
# Exporting lat long as it is now finalized
Lat_Long_df.to_csv("Lat_Long_cleaned.csv", index=False)

In [24]:
### Working on Econ dataframe
# Renaming columns to match other datasets
Econ_df = Econ_df.rename(columns={"FIPS_code":"fips","State":"ST"})
Econ_df

,fips,ST,Area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,...,Civilian_labor_force_2020,Employed_2020,Unemployed_2020,Unemployment_rate_2020,Civilian_labor_force_2021,Employed_2021,Unemployed_2021,Unemployment_rate_2021,Median_Household_Income_2020,Med_HH_Income_Percent_of_State_Total_2020
0,0,US,United States,NaN,NaN,NaN,"142,601,576","136,904,853","5,696,723",4.0,...,"161,207,103","148,183,367","13,023,736",8.1,"161,758,337","153,099,687","8,658,650",5.4,"67,340",NaN
1,1000,AL,Alabama,NaN,NaN,NaN,"2,147,173","2,047,731","99,442",4.6,...,"2,268,309","2,121,149","147,160",6.5,"2,246,993","2,169,721","77,272",3.4,"53,958",100.0
2,1001,AL,"Autauga County, AL",2.0,2.0,1.0,"21,861","20,971",890,4.1,...,"26,405","24,985","1,420",5.4,"26,341","25,599",742,2.8,"67,565",125.2
3,1003,AL,"Baldwin County, AL",3.0,2.0,1.0,"69,979","67,370","2,609",3.7,...,"98,910","92,751","6,159",6.2,"99,427","96,481","2,946",3.0,"71,135",131.8
4,1005,AL,"Barbour County, AL",6.0,6.0,0.0,"11,449","10,812",637,5.6,...,"8,680","8,004",676,7.8,"8,197","7,728",469,5.7,"38,866",72.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3193,56037,WY,"Sweetwater County, WY",5.0,8.0,0.0,"20,623","19,822",801,3.9,...,"20,503","18,998","1,505",7.3,"19,449","18,360","1,089",5.6,"70,583",104.9
3194,56039,WY,"Teton County, WY",7.0,8.0,0.0,"14,126","13,791",335,2.4,...,"15,138","14,251",887,5.9,"15,886","15,375",511,3.2,"92,488",137.5
3195,56041,WY,"Uinta County, WY",7.0,8.0,0.0,"10,414","10,004",410,3.9,...,"9,102","8,532",570,6.3,"8,832","8,389",443,5.0,"71,246",105.9
3196,56043,WY,"Washakie County, WY",7.0,11.0,0.0,"4,287","4,093",194,4.5,...,"3,945","3,739",206,5.2,"3,978","3,819",159,4.0,"58,532",87.0


In [25]:
# Choosing the columns I want from Econ
Econ_df = Econ_df[["fips","ST","Unemployment_rate_2021","Median_Household_Income_2020"]]
Econ_df

,fips,ST,Unemployment_rate_2021,Median_Household_Income_2020
0,0,US,5.4,"67,340"
1,1000,AL,3.4,"53,958"
2,1001,AL,2.8,"67,565"
3,1003,AL,3.0,"71,135"
4,1005,AL,5.7,"38,866"
...,...,...,...,...
3193,56037,WY,5.6,"70,583"
3194,56039,WY,3.2,"92,488"
3195,56041,WY,5.0,"71,246"
3196,56043,WY,4.0,"58,532"


In [26]:
# Checking datatypes of columns
Econ_df.dtypes

fips                              int64
ST                               object
Unemployment_rate_2021          float64
Median_Household_Income_2020     object
dtype: object

In [27]:
# Converting median income to integer part 1
Econ_df.replace(",","", regex=True, inplace=True)
Econ_df

C:\Users\josep\anaconda3\envs\mlenv\lib\site-packages\pandas\core\frame.py:5244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


,fips,ST,Unemployment_rate_2021,Median_Household_Income_2020
0,0,US,5.4,67340
1,1000,AL,3.4,53958
2,1001,AL,2.8,67565
3,1003,AL,3.0,71135
4,1005,AL,5.7,38866
...,...,...,...,...
3193,56037,WY,5.6,70583
3194,56039,WY,3.2,92488
3195,56041,WY,5.0,71246
3196,56043,WY,4.0,58532


In [28]:
# Dropping null values from median income and US row
Econ_df = Econ_df.dropna(subset=['Median_Household_Income_2020'])
Econ_df = Econ_df.drop(labels=0, axis=0)
Econ_df

,fips,ST,Unemployment_rate_2021,Median_Household_Income_2020
1,1000,AL,3.4,53958
2,1001,AL,2.8,67565
3,1003,AL,3.0,71135
4,1005,AL,5.7,38866
5,1007,AL,3.5,50907
...,...,...,...,...
3193,56037,WY,5.6,70583
3194,56039,WY,3.2,92488
3195,56041,WY,5.0,71246
3196,56043,WY,4.0,58532


In [29]:
# Dropping the rest of the states (I want only county information)
Econ_df = Econ_df[Econ_df['fips'] % 1000 != 0]
Econ_df

,fips,ST,Unemployment_rate_2021,Median_Household_Income_2020
2,1001,AL,2.8,67565
3,1003,AL,3.0,71135
4,1005,AL,5.7,38866
5,1007,AL,3.5,50907
6,1009,AL,2.4,55203
...,...,...,...,...
3193,56037,WY,5.6,70583
3194,56039,WY,3.2,92488
3195,56041,WY,5.0,71246
3196,56043,WY,4.0,58532


In [30]:
# Converting datatype of median income into integer, part 2
Econ_df['Median_Household_Income_2020'] = Econ_df['Median_Household_Income_2020'].astype(str).astype(int)
Econ_df.dtypes

C:\Users\josep\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


fips                              int64
ST                               object
Unemployment_rate_2021          float64
Median_Household_Income_2020      int32
dtype: object

In [31]:
# Exporting Econ dataframe as it is now finalized
Econ_df.to_csv("Econ_cleaned.csv", index=False)
Econ_df

,fips,ST,Unemployment_rate_2021,Median_Household_Income_2020
2,1001,AL,2.8,67565
3,1003,AL,3.0,71135
4,1005,AL,5.7,38866
5,1007,AL,3.5,50907
6,1009,AL,2.4,55203
...,...,...,...,...
3193,56037,WY,5.6,70583
3194,56039,WY,3.2,92488
3195,56041,WY,5.0,71246
3196,56043,WY,4.0,58532


In [32]:
### MERGING THE FOUR DATASETS
# Merging vax and lat long
Coord_Vax_df = Lat_Long_df.merge(Latest_Vax_df, how="left")
Coord_Vax_df

,fips,ST,lat,lng,population,Recip_County,Vax_Pct
0,6037,CA,34.3209,-118.2247,10040682,Los Angeles County,73.6
1,17031,IL,41.8401,-87.8168,5169517,Cook County,73.7
2,48201,TX,29.8578,-95.3936,4680609,Harris County,64.7
3,4013,AZ,33.3490,-112.4915,4412779,Maricopa County,58.8
4,6073,CA,33.0343,-116.7350,3323970,San Diego County,77.9
...,...,...,...,...,...,...,...
3138,35021,NM,35.8579,-103.8202,432,Harding County,60.3
3139,31117,NE,41.5682,-101.0604,420,McPherson County,14.8
3140,48261,TX,26.9285,-97.7017,391,Kenedy County,54.0
3141,48269,TX,33.6165,-100.2558,279,King County,21.3


In [33]:
# Merging the above df with deaths
Coord_Vax_Deaths_df = Coord_Vax_df.merge(Latest_Deaths_df, how="left")
Coord_Vax_Deaths_df

,fips,ST,lat,lng,population,Recip_County,Vax_Pct,date,county,cases,deaths
0,6037,CA,34.3209,-118.2247,10040682,Los Angeles County,73.6,2022-09-14,Los Angeles,3434458.0,33414.0
1,17031,IL,41.8401,-87.8168,5169517,Cook County,73.7,2022-09-14,Cook,1392834.0,15312.0
2,48201,TX,29.8578,-95.3936,4680609,Harris County,64.7,2022-09-14,Harris,1208024.0,11270.0
3,4013,AZ,33.3490,-112.4915,4412779,Maricopa County,58.8,2022-09-14,Maricopa,1421340.0,17865.0
4,6073,CA,33.0343,-116.7350,3323970,San Diego County,77.9,2022-09-14,San Diego,986208.0,5474.0
...,...,...,...,...,...,...,...,...,...,...,...
3138,35021,NM,35.8579,-103.8202,432,Harding County,60.3,2022-09-14,Harding,94.0,4.0
3139,31117,NE,41.5682,-101.0604,420,McPherson County,14.8,2022-09-14,McPherson,74.0,1.0
3140,48261,TX,26.9285,-97.7017,391,Kenedy County,54.0,2022-09-14,Kenedy,97.0,2.0
3141,48269,TX,33.6165,-100.2558,279,King County,21.3,2022-09-14,King,53.0,0.0


In [34]:
# Seeing if the merges resulted in null values
Coord_Vax_Deaths_df.isna().sum()

fips           0
ST             0
lat            0
lng            0
population     0
              ..
Vax_Pct       18
date          11
county        11
cases         11
deaths        11
Length: 11, dtype: int64

In [35]:
# Dropping the null values that the merges generated
Coord_Vax_Deaths_df = Coord_Vax_Deaths_df.dropna(subset=['Vax_Pct'])
Coord_Vax_Deaths_df = Coord_Vax_Deaths_df.dropna(subset=['cases'])
Coord_Vax_Deaths_df

,fips,ST,lat,lng,population,Recip_County,Vax_Pct,date,county,cases,deaths
0,6037,CA,34.3209,-118.2247,10040682,Los Angeles County,73.6,2022-09-14,Los Angeles,3434458.0,33414.0
1,17031,IL,41.8401,-87.8168,5169517,Cook County,73.7,2022-09-14,Cook,1392834.0,15312.0
2,48201,TX,29.8578,-95.3936,4680609,Harris County,64.7,2022-09-14,Harris,1208024.0,11270.0
3,4013,AZ,33.3490,-112.4915,4412779,Maricopa County,58.8,2022-09-14,Maricopa,1421340.0,17865.0
4,6073,CA,33.0343,-116.7350,3323970,San Diego County,77.9,2022-09-14,San Diego,986208.0,5474.0
...,...,...,...,...,...,...,...,...,...,...,...
3138,35021,NM,35.8579,-103.8202,432,Harding County,60.3,2022-09-14,Harding,94.0,4.0
3139,31117,NE,41.5682,-101.0604,420,McPherson County,14.8,2022-09-14,McPherson,74.0,1.0
3140,48261,TX,26.9285,-97.7017,391,Kenedy County,54.0,2022-09-14,Kenedy,97.0,2.0
3141,48269,TX,33.6165,-100.2558,279,King County,21.3,2022-09-14,King,53.0,0.0


In [36]:
# Merging above dataframe with Econ
Coord_Vax_Deaths_Econ_df = Coord_Vax_Deaths_df.merge(Econ_df, how="left")
Coord_Vax_Deaths_Econ_df

,fips,ST,lat,lng,population,Recip_County,Vax_Pct,date,county,cases,deaths,Unemployment_rate_2021,Median_Household_Income_2020
0,6037,CA,34.3209,-118.2247,10040682,Los Angeles County,73.6,2022-09-14,Los Angeles,3434458.0,33414.0,8.9,75624
1,17031,IL,41.8401,-87.8168,5169517,Cook County,73.7,2022-09-14,Cook,1392834.0,15312.0,7.0,71611
2,48201,TX,29.8578,-95.3936,4680609,Harris County,64.7,2022-09-14,Harris,1208024.0,11270.0,6.5,61906
3,4013,AZ,33.3490,-112.4915,4412779,Maricopa County,58.8,2022-09-14,Maricopa,1421340.0,17865.0,4.5,71799
4,6073,CA,33.0343,-116.7350,3323970,San Diego County,77.9,2022-09-14,San Diego,986208.0,5474.0,6.5,87126
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3111,35021,NM,35.8579,-103.8202,432,Harding County,60.3,2022-09-14,Harding,94.0,4.0,4.6,39913
3112,31117,NE,41.5682,-101.0604,420,McPherson County,14.8,2022-09-14,McPherson,74.0,1.0,1.6,54876
3113,48261,TX,26.9285,-97.7017,391,Kenedy County,54.0,2022-09-14,Kenedy,97.0,2.0,6.0,45980
3114,48269,TX,33.6165,-100.2558,279,King County,21.3,2022-09-14,King,53.0,0.0,0.9,78210


In [37]:
# Checking to see if merge resulted in null values
pd.set_option('display.max_rows', 15)
Coord_Vax_Deaths_Econ_df.isna().sum()

fips                            0
ST                              0
lat                             0
lng                             0
population                      0
Recip_County                    0
Vax_Pct                         0
date                            0
county                          0
cases                           0
deaths                          0
Unemployment_rate_2021          0
Median_Household_Income_2020    0
dtype: int64

In [38]:
# Creating Cases and Deaths per 100,000 statistics using population and cases/deaths columns
Coord_Vax_Deaths_Econ_df['cases_100000'] = Coord_Vax_Deaths_Econ_df.cases / Coord_Vax_Deaths_Econ_df.population * 100000
Coord_Vax_Deaths_Econ_df['deaths_100000'] = Coord_Vax_Deaths_Econ_df.deaths / Coord_Vax_Deaths_Econ_df.population * 100000
Coord_Vax_Deaths_Econ_df

,fips,ST,lat,lng,population,Recip_County,Vax_Pct,date,county,cases,deaths,Unemployment_rate_2021,Median_Household_Income_2020,cases_100000,deaths_100000
0,6037,CA,34.3209,-118.2247,10040682,Los Angeles County,73.6,2022-09-14,Los Angeles,3434458.0,33414.0,8.9,75624,34205.425488,332.786159
1,17031,IL,41.8401,-87.8168,5169517,Cook County,73.7,2022-09-14,Cook,1392834.0,15312.0,7.0,71611,26943.213457,296.197885
2,48201,TX,29.8578,-95.3936,4680609,Harris County,64.7,2022-09-14,Harris,1208024.0,11270.0,6.5,61906,25809.120138,240.780633
3,4013,AZ,33.3490,-112.4915,4412779,Maricopa County,58.8,2022-09-14,Maricopa,1421340.0,17865.0,4.5,71799,32209.634790,404.846923
4,6073,CA,33.0343,-116.7350,3323970,San Diego County,77.9,2022-09-14,San Diego,986208.0,5474.0,6.5,87126,29669.581855,164.682593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3111,35021,NM,35.8579,-103.8202,432,Harding County,60.3,2022-09-14,Harding,94.0,4.0,4.6,39913,21759.259259,925.925926
3112,31117,NE,41.5682,-101.0604,420,McPherson County,14.8,2022-09-14,McPherson,74.0,1.0,1.6,54876,17619.047619,238.095238
3113,48261,TX,26.9285,-97.7017,391,Kenedy County,54.0,2022-09-14,Kenedy,97.0,2.0,6.0,45980,24808.184143,511.508951
3114,48269,TX,33.6165,-100.2558,279,King County,21.3,2022-09-14,King,53.0,0.0,0.9,78210,18996.415771,0.000000


In [39]:
# Exporting finalized df with all data
Coord_Vax_Deaths_Econ_df.to_csv("all_data_merged.csv", index=False)